<a href="https://colab.research.google.com/github/Ankur3107/GitHub-Bugs-Prediction-Challenge/blob/main/models/5Fold_Model_Electra_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.1MB 659kB/s 
     |████████████████████████████████| 1.1MB 4.7MB/s 
     |████████████████████████████████| 890kB 8.2MB/s 
     |████████████████████████████████| 3.0MB 6.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b1426c2dd841ffa6b86df5c159aaf63de72eef5d0d5f96c865fbeeec1110c19a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
!unzip ./Embold_Participant\'s_Dataset.zip -d Dataset

--2020-10-16 16:37:39--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.64.88
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.64.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘Embold_Participant's_Dataset.zip’

Embold_Participant' 100%[===================>]  97.58M  12.8MB/s    in 9.3s    

2020-10-16 16:37:49 (10.4 MB/s) - ‘Embold_Participant's_Dataset.zip’ saved [102320961/102320961]

Archive:  ./Embold_Participant's_Dataset.zip
   creating: Dataset/Embold_Participant's_Dataset/
  inflating: Dataset/Embold_Participant's_Dataset/sample submission.csv  
  inflating: Dataset/__MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Dataset/Embold_Participant's_Dataset/embold_train_extra

In [3]:
cd "Dataset/Embold_Participant's_Dataset/"

/content/Dataset/Embold_Participant's_Dataset


In [4]:
import numpy as np
import pandas as pd

In [5]:
train_df = pd.read_json("embold_train.json").reset_index(drop=True)
test_df = pd.read_json("embold_test.json").reset_index(drop=True)
train_ex_df = pd.read_json("embold_train_extra.json")

In [6]:
train_data = train_df.append(train_ex_df)
test_df['text'] = test_df['title']+' '+test_df['body']
train_data['text'] = train_data['title']+' '+train_data['body']

In [7]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import *
from transformers import pipeline
from pprint import pprint
from tensorflow import keras

In [8]:
@dataclass
class Config:
    MAX_LEN = 320
    BATCH_SIZE = 16  # per TPU core
    TOTAL_STEPS = 2000  # thats approx 4 epochs
    EVALUATE_EVERY = 200
    LR = 1e-5
    PRETRAINED_MODEL = "google/electra-small-discriminator"  # huggingface bert model


transformer_flags = Config()


def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print("Running on TPU ", tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = transformer_flags.BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.21.136.122:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.21.136.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.21.136.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [9]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )

    return np.array(enc_di["input_ids"])

#tokenizer = AutoTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = BertTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
#tokenizer = AlbertTokenizer.from_pretrained(transformer_flags.PRETRAINED_MODEL, use_fast=True)
#tokenizer = RobertaTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)
tokenizer = ElectraTokenizerFast.from_pretrained(transformer_flags.PRETRAINED_MODEL)

In [18]:
def build_model():
  input_layer = tf.keras.layers.Input((transformer_flags.MAX_LEN,), dtype=tf.int32)
  bert_model = TFAutoModel.from_pretrained(transformer_flags.PRETRAINED_MODEL)
  output_layer = bert_model(input_layer) 
  pooled_output = tf.keras.layers.GlobalAveragePooling1D()(output_layer[0])
  output = tf.keras.layers.Dense(3, activation='softmax')(pooled_output)
  classifier_model = tf.keras.Model(input_layer, output)

  optimizer = tf.keras.optimizers.Adam(learning_rate=transformer_flags.LR)
  classifier_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  classifier_model.summary()
    
  return classifier_model

In [11]:
train = train_data[0:150000]
validation = train_data[150000:200000]
train.shape, validation.shape

((150000, 4), (50000, 4))

In [12]:
# data preparation
X_train = regular_encode(train.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_train = train.label.values

print(X_train.shape, y_train.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(150000, 320) (150000,)


In [13]:
# validation data preparation
X_val = regular_encode(validation.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
y_val = validation.label.values

print(X_val.shape, X_val.shape)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(50000, 320) (50000, 320)


In [14]:
X_test = regular_encode(test_df.text.values.tolist(), tokenizer, maxlen=transformer_flags.MAX_LEN)
X_test.shape

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(30000, 320)

In [15]:
from sklearn.model_selection import StratifiedKFold
SEED = 3107
LABEL_SMOOTHING = 0.1

In [16]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=SEED)

In [19]:
oof_fold_val = np.zeros((X_train.shape[0],3))
preds = np.zeros((X_test.shape[0],3))
validation_preds = np.zeros((X_val.shape[0],3))

for fold,(idxT,idxV) in enumerate(skf.split(X_train,y_train)):
  print('#'*25)
  print('### FOLD %i'%(fold+1))
  print('#'*25)

  tf.keras.backend.clear_session()
  with strategy.scope():
    model = build_model()

  inpT = X_train[idxT,]
  targetT = y_train[idxT,]
  inpV = X_train[idxV,]
  targetV = y_train[idxV,]

  model.fit(inpT, targetT,epochs=2, batch_size=global_batch_size, verbose=1,
            validation_data=(inpV, targetV), shuffle=True)
  
  fiVal_probs = model.predict(inpV, batch_size=global_batch_size, verbose=1)

  oof_fold_val[idxV,] = fiVal_probs
  validation_probs = model.predict(X_val, batch_size=global_batch_size, verbose=1)
  test_probs = model.predict(X_test, batch_size=global_batch_size, verbose=1)

  preds += test_probs/skf.n_splits
  validation_preds += validation_probs/skf.n_splits

  np.save('validation_probs'+str(fold+1)+'.npy', validation_probs)
  np.save('test_probs'+str(fold+1)+'.npy', test_probs)


#########################
### FOLD 1
#########################


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the model checkpoint at google/electra-small-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 320)]             0         
_________________________________________________________________
tf_electra_model (TFElectraM ((None, 320, 256),)       13483008  
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 771       
Total params: 13,483,779
Trainable params: 13,483,779
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  2/938 [..............................] - ETA: 3:48:30 - loss: 1.0959 - accuracy: 0.3555WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0082s vs `on_train_batch_end` time: 0.0992s). Check your callbacks.


...

938/938 [==============================] - ETA: 0s - loss: 0.6075 - accuracy: 0.7516WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0035s vs `on_test_batch_end` time: 0.0352s). Check your callbacks.


938/938 [==============================] - 176s 187ms/step - loss: 0.6075 - accuracy: 0.7516 - val_loss: 0.4952 - val_accuracy: 0.8136
Epoch 2/2
  2/235 [..............................] - ETA: 11:08WARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0035s vs `on_predict_batch_end` time: 0.0337s). Check your callbacks.


  1/391 [..............................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0036s vs `on_predict_batch_end` time: 0.0310s). Check your callbacks.


  1/235 [..............................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0029s vs `on_predict_batch_end` time: 0.0312s). Check your callbacks.


235/235 [==============================] - 8s 35ms/step


In [23]:
preds_labels = preds.argmax(axis=1)

In [22]:
validation_preds_labels = validation_preds.argmax(axis=1)

In [24]:
from sklearn.metrics import classification_report, accuracy_score

In [26]:
print(classification_report(y_val, validation_preds_labels))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85     22378
           1       0.84      0.86      0.85     23000
           2       0.63      0.50      0.56      4622

    accuracy                           0.82     50000
   macro avg       0.77      0.74      0.75     50000
weighted avg       0.82      0.82      0.82     50000



In [27]:
print(accuracy_score(y_val, validation_preds_labels))

0.82292


In [28]:
np.save('y_val.npy', y_val)